<a href="https://colab.research.google.com/github/xoro-o/nn_baisc/blob/main/basic_nn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # Imports  
 import torch
 import torch.nn as nn
 import torch.optim as optim # sgd adam etc
 import torch.nn.functional as F # all function which dont take parameters such as ReLU 
 from torch.utils.data import DataLoader
 import torchvision.datasets as datasets
 import torchvision.transforms as transforms
 

In [4]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__() 
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)

  def forward(self,x):
    x =   F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


model = NN(784,10)
x = torch.randn(64,784)
print(model(x).shape)



torch.Size([64, 10])


In [5]:
# set device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs =  1


In [6]:
# Load data
train_dataset = datasets.MNIST(root='dataset/',train=True,transform = transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform = transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [11]:
# initialize network
model = NN(input_size = input_size,num_classes = num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)



In [12]:
#train network
for epoch in range(num_epochs):
  for batch,(data, target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)
    
    data = data.reshape(data.shape[0],-1)
    
    pred = model(data)
    loss = loss_fn(pred,target)

    
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    
    


In [13]:
def check_accuracy(loader,model):
  num_correct = 0
  num_sample = 0 
  model.eval()
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y=y.to(device=device)
      x = x.reshape(x.shape[0],-1)

      scores = model(x)
      _,pred = scores.max(dim=1)
      num_correct += (pred==y).sum()
      num_sample += pred.size(0)
    print(f'accuracy = {float(num_correct)/float(num_sample)*100:.2f}')

In [17]:
check_accuracy(test_loader,model)

accuracy = 0.9352
